In [1]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt


%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
botswana = pd.read_table('botswana.tsv')

Данные опроса 4361 женщин из Ботсваны:

$ceb$ - сколько детей она родила

$age$ - возраст

$educ$ - длительность получения образования

$religion$ - религиозная принадлежность

$idlnchld$ - идеальное, по её мнению, количество детей в семье

$evermarr$ - была ли она когда-нибудь замужем

$agefm$ - возраст первого замужества

$heduc$ - длительность получения образования мужем

$knowmeth$ - знает ли она о методах контрацепции

$usemeth$ - использует ли она методы контрацепции

$urban$ - живёт ли она в городе

$electric, radio, tv, bicycle$ - есть ли у неё электричество, радио, телевизор и велосипед

In [3]:
botswana.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0


In [4]:
np.unique(botswana.religion)

array(['catholic', 'other', 'protestant', 'spirit'], dtype=object)

In [5]:
botswana.religion.isnull().sum()

0

In [6]:
botswana.describe()

,ceb,age,educ,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
count,4361.000000,4361.000000,4361.000000,4241.000000,4354.000000,4290.000000,4361.000000,2079.000000,1956.000000,4361.000000,4358.000000,4359.000000,4359.000000,4358.000000
mean,2.441642,27.405182,5.855996,4.615892,0.963252,0.577622,0.476726,20.686388,5.144683,0.516625,0.140202,0.701766,0.092911,0.275815
std,2.406861,8.685233,3.927075,2.219303,0.188164,0.493996,0.499515,5.002383,4.803028,0.499781,0.347236,0.457535,0.290341,0.446975
min,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,20.000000,3.000000,3.000000,1.000000,0.000000,0.000000,17.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,26.000000,7.000000,4.000000,1.000000,1.000000,0.000000,20.000000,6.000000,1.000000,0.000000,1.000000,0.000000,0.000000
75%,4.000000,33.000000,8.000000,6.000000,1.000000,1.000000,1.000000,23.000000,8.000000,1.000000,0.000000,1.000000,0.000000,1.000000
max,13.000000,49.000000,20.000000,20.000000,1.000000,1.000000,1.000000,46.000000,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
botswana_clear  = botswana.dropna()

In [8]:
print(botswana.shape, botswana_clear.shape)

(4361, 15) (1834, 15)


In [9]:
botswana.agefm.fillna(0, inplace = True)

In [10]:
botswana['nevermarr'] = botswana['agefm'].apply(lambda x : 1 if x == 0 else 0)

In [11]:
botswana.drop('evermarr', axis = 1, inplace = True)

In [12]:
botswana.loc[botswana.nevermarr == 1, 'heduc'] = -1

In [13]:
sum(botswana.heduc.isna())

123

In [14]:
botswana.idlnchld.fillna(-1, inplace = True)
botswana.heduc.fillna(-2, inplace = True)
botswana.usemeth.fillna(-1, inplace = True)

In [15]:
botswana['idlnchld_noans'] = botswana['idlnchld'].apply(lambda x : 1 if x == -1 else 0)
botswana['heduc_noans'] = botswana.heduc.apply(lambda x: 1 if x == -2 else 0)
botswana['usemeth_noans'] = botswana.usemeth.apply(lambda x : 1 if x == -1 else 0)

In [16]:
botswana.head()

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr,idlnchld_noans,heduc_noans,usemeth_noans
0,0,18,10,catholic,4.0,1.0,1.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0,0,0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0,0,0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,-1.0,1,1.0,1.0,1.0,1.0,1,0,0,0
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0,0,0,0


In [17]:
botswana.dropna(inplace = True)

In [18]:
botswana.shape

(4348, 18)

In [19]:
4348*18

78264

In [20]:
for name in botswana.columns:
    print(name, end = ' + ')

ceb + age + educ + religion + idlnchld + knowmeth + usemeth + agefm + heduc + urban + electric + radio + tv + bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans + 

In [21]:
model = smf.ols( 'ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth +'\
                        'agefm + heduc + urban + electric + radio + tv + bicycle +'\
                                ' nevermarr + idlnchld_noans + heduc_noans + usemeth_noans',
                data = botswana)

fitted = model.fit()
print (fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Sat, 21 Jul 2018   Prob (F-statistic):               0.00
Time:                        17:02:52   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [22]:
print ('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted.resid, fitted.model.exog)[1])

Breusch-Pagan test: p=0.000000


Ошибки гетероскедастичны, значит, значимость признаков может определяться неверно. Сделаем поправку Уайта:

In [23]:
model_WHITE = smf.ols( 'ceb ~ age + educ + religion + idlnchld + knowmeth + usemeth +'\
                        'agefm + heduc + urban + electric + radio + tv + bicycle +'\
                          'nevermarr + idlnchld_noans + heduc_noans + usemeth_noans',
                data = botswana)

fitted_WHITE = model_WHITE.fit(cov_type='HC1')
print (fitted_WHITE.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     345.0
Date:                Sat, 21 Jul 2018   Prob (F-statistic):               0.00
Time:                        17:02:52   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:                  HC1                                         
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

In [24]:
model3 = smf.ols( 'ceb ~ age + educ + idlnchld + knowmeth + usemeth +'\
                        'agefm + heduc + urban + electric + bicycle +' \
                         'nevermarr + idlnchld_noans + heduc_noans + usemeth_noans',
                data = botswana)

fitted3 = model3.fit()
print (fitted3.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     559.5
Date:                Sat, 21 Jul 2018   Prob (F-statistic):               0.00
Time:                        17:02:52   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.198     -5.

In [25]:
print ('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted3.resid, fitted3.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [26]:
botswana1 = botswana.drop(columns = ['religion', 'radio', 'tv'])

In [27]:
model4 = smf.ols( 'ceb ~ age + educ + idlnchld + knowmeth + usemeth +'\
                        'agefm + heduc + urban + electric + bicycle +'\
                         'nevermarr + idlnchld_noans + heduc_noans + usemeth_noans',
                data = botswana1)

fitted4 = model4.fit(cov_type='HC1')
print (fitted4.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Sat, 21 Jul 2018   Prob (F-statistic):               0.00
Time:                        17:02:52   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

In [28]:
print ("F=%f, p=%f, k1=%f" % model.fit().compare_f_test(model4.fit()))

F=0.919236, p=0.467231, k1=5.000000


In [29]:
model5 = smf.ols( 'ceb ~ age + educ + idlnchld + knowmeth +'\
                        'agefm + heduc + urban + electric + bicycle +'\
                         'nevermarr + idlnchld_noans + heduc_noans',
                data = botswana1)

fitted5 = model5.fit()
print (fitted5.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.629
Model:                            OLS   Adj. R-squared:                  0.628
Method:                 Least Squares   F-statistic:                     611.3
Date:                Sat, 21 Jul 2018   Prob (F-statistic):               0.00
Time:                        17:02:53   Log-Likelihood:                -7825.7
No. Observations:                4348   AIC:                         1.568e+04
Df Residuals:                    4335   BIC:                         1.576e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.1931      0.202     -5.

In [30]:
F, p, k1 = model4.fit().compare_f_test(model5.fit())

In [31]:
p

3.155200948040263e-40